In [6]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
params = {
	"latitude": 38.6968,
	"longitude": -9.4215,
	"start_date": "2016-01-01",
	"end_date": "2017-12-31",
	"hourly": ["temperature_500hPa", "relative_humidity_2m"],
	"models": "ecmwf_ifs025",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_950hPa = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_950hPa"] = hourly_temperature_950hPa
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)

Coordinates: 38.75°N -9.25°E
Elevation: 17.0 m asl
Timezone difference to GMT+0: 0s

Hourly data
                            date  temperature_950hPa  relative_humidity_2m
0     2016-01-01 00:00:00+00:00                 NaN                   NaN
1     2016-01-01 01:00:00+00:00                 NaN                   NaN
2     2016-01-01 02:00:00+00:00                 NaN                   NaN
3     2016-01-01 03:00:00+00:00                 NaN                   NaN
4     2016-01-01 04:00:00+00:00                 NaN                   NaN
...                         ...                 ...                   ...
17539 2017-12-31 19:00:00+00:00                 NaN                   NaN
17540 2017-12-31 20:00:00+00:00                 NaN                   NaN
17541 2017-12-31 21:00:00+00:00                 NaN                   NaN
17542 2017-12-31 22:00:00+00:00                 NaN                   NaN
17543 2017-12-31 23:00:00+00:00                 NaN                   NaN

[17544 rows x